# Finite State Machines in Digital Design

*From state diagrams to hardware implementation*

---

## Prerequisites

This tutorial assumes you have completed:

- **[D Flip-Flops in Digital Design](d_flipflop_tutorial.ipynb)** — understanding edge-triggered storage and clock signals
- **[Timing Diagrams in Digital Design](timing_diagrams_tutorial.ipynb)** — reading waveforms and understanding signal timing

You should also be familiar with:

- **Binary numbers** and basic Boolean algebra (AND, OR, NOT)
- **Logic gates** and how to combine them into circuits

---

## What is a Finite State Machine?

Finite State Machines (FSMs) are fundamental building blocks in digital design. They model systems that can be in one of a finite number of states, transitioning between states based on inputs.

An FSM consists of:

- **States**: A finite set of conditions the system can be in
- **Inputs**: External signals that influence state transitions
- **Outputs**: Signals produced by the machine
- **Transitions**: Rules for moving between states based on inputs
- **Initial State**: The starting state when the system powers on

Think of a traffic light: it cycles through states (Red → Green → Yellow → Red), with timing inputs controlling transitions and the light color as output.

---

## State Diagrams: Visualizing FSMs

State diagrams use circles for states and arrows for transitions. Let's visualize a simple two-state FSM that toggles between ON and OFF based on a button press.

This is a **Moore machine** — the output (LED on/off) depends only on the current state.

**Color convention:** Green = output HIGH (LED on), Grey = output LOW (LED off)

In [ ]:
from graphviz import Digraph
from IPython.display import display, SVG

dot = Digraph('toggle_fsm')
dot.attr(rankdir='LR')
dot.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='12', width='0.8')
dot.attr('edge', fontname='Arial', fontsize='10')

# States with outputs shown (Moore machine notation)
dot.node('OFF', 'OFF\nLED=0', fillcolor='#6b7280', fontcolor='white')
dot.node('ON', 'ON\nLED=1', fillcolor='#10b981', fontcolor='white')

# Initial state indicator
dot.node('start', '', shape='point', width='0')
dot.edge('start', 'OFF')

# Transitions
dot.edge('OFF', 'ON', label='button=1', color='#3b82f6', fontcolor='#3b82f6', penwidth='2')
dot.edge('ON', 'OFF', label='button=1', color='#3b82f6', fontcolor='#3b82f6', penwidth='2')

# Self-loops
dot.edge('OFF', 'OFF', label='button=0', color='#9ca3af', fontcolor='#6b7280')
dot.edge('ON', 'ON', label='button=0', color='#9ca3af', fontcolor='#6b7280')

display(SVG(dot.pipe(format='svg')))

This FSM has:
- **2 states**: OFF, ON
- **1 input**: button (0 or 1)
- **1 output**: LED (0=off, 1=on) — determined by the **current state**
  - State OFF → LED = 0
  - State ON → LED = 1
- **Transitions**: Toggle state when button=1, stay in current state when button=0
- **Initial state**: OFF (indicated by the incoming arrow with no source state)

**Note:** This is a Moore machine because the output depends only on which state we're in, not on the input value.

### State Table

The same FSM can be represented as a table showing all possible transitions:

| Current State | Input (button) | Next State | Output (LED) |
|--------------|----------------|------------|--------------|
| OFF          | 0              | OFF        | 0            |
| OFF          | 1              | ON         | 0            |
| ON           | 0              | ON         | 1            |
| ON           | 1              | OFF        | 1            |

**How to read this table (Moore machine):**

- **Output column**: Shows the LED value *while in the current state* (before the clock edge)
- **Next State column**: Shows where we go *after the clock edge* based on the input
- **To find output after transition**: Look up the next state in the table's "Current State" column

**Example walkthrough:**

- **Row 1**: In state OFF (LED=0). Input button=0. Stay in OFF (LED remains 0).

- **Row 2**: In state OFF (LED=0). Input button=1. Transition to ON state.
  - *After clock edge*: Look at rows 3-4 where "Current State" = ON → LED becomes 1

- **Row 3**: In state ON (LED=1). Input button=0. Stay in ON (LED remains 1).

- **Row 4**: In state ON (LED=1). Input button=1. Transition to OFF state.
  - *After clock edge*: Look at rows 1-2 where "Current State" = OFF → LED becomes 0

**Key insight:** Each state's output is shown in *that state's rows*. To trace a complete transition, you need to look up both the current state's row (for current output) and the next state's rows (for output after transition).

---

## Moore vs Mealy Machines

There are two fundamental types of FSMs, differing in how outputs are determined:

### Moore Machine
- **Output depends only on the current state**
- Output is associated with states
- More states may be needed, but timing is simpler
- Output changes only on clock edges (synchronous)

### Mealy Machine
- **Output depends on current state AND inputs**
- Output is associated with transitions
- Fewer states possible, but more complex timing
- Output can change asynchronously with inputs

Let's visualize both for the same problem: **detecting when input X has been 1 for two consecutive clock cycles**. Output Z goes HIGH when we've seen two 1s in a row.

**Color convention:** Blue = pattern not found yet (Z=0), Green = two consecutive 1s detected (Z=1)

In [ ]:
from graphviz import Digraph
from IPython.display import display, HTML

# === MOORE MACHINE ===
moore = Digraph('moore')
moore.attr(rankdir='LR', label='Moore Machine\n(Output depends on state only)\n3 states needed', labelloc='t', fontsize='14')
moore.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='11', width='0.7')
moore.attr('edge', fontname='Arial', fontsize='9')

moore.node('S0', 'S0\nZ=0', fillcolor='#3b82f6', fontcolor='white')
moore.node('S1', 'S1\nZ=0', fillcolor='#3b82f6', fontcolor='white')
moore.node('S2', 'S2\nZ=1', fillcolor='#10b981', fontcolor='white')
moore.node('start_m', '', shape='point', width='0')
moore.edge('start_m', 'S0')
moore.edge('S0', 'S1', label='X=1', penwidth='1.5')
moore.edge('S1', 'S2', label='X=1', penwidth='1.5')
moore.edge('S2', 'S2', label='X=1', penwidth='1.5')
moore.edge('S0', 'S0', label='X=0', color='#9ca3af', fontcolor='#6b7280')
moore.edge('S1', 'S0', label='X=0', color='#9ca3af', fontcolor='#6b7280')
moore.edge('S2', 'S0', label='X=0', color='#9ca3af', fontcolor='#6b7280')

# === MEALY MACHINE ===
mealy = Digraph('mealy')
mealy.attr(rankdir='LR', label='Mealy Machine\n(Output depends on state AND input)\n2 states needed', labelloc='t', fontsize='14')
mealy.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='11', width='0.7')
mealy.attr('edge', fontname='Arial', fontsize='9')

mealy.node('M0', 'S0', fillcolor='#3b82f6', fontcolor='white')
mealy.node('M1', 'S1', fillcolor='#3b82f6', fontcolor='white')
mealy.node('start_e', '', shape='point', width='0')
mealy.edge('start_e', 'M0')
mealy.edge('M0', 'M1', label='X=1 / Z=0', penwidth='1.5')
mealy.edge('M1', 'M1', label='X=1 / Z=1', color='#10b981', fontcolor='#10b981', penwidth='2')
mealy.edge('M0', 'M0', label='X=0 / Z=0', color='#9ca3af', fontcolor='#6b7280')
mealy.edge('M1', 'M0', label='X=0 / Z=0', color='#9ca3af', fontcolor='#6b7280')

# Get SVG strings and embed in a single HTML output
moore_svg = moore.pipe(format='svg').decode('utf-8')
mealy_svg = mealy.pipe(format='svg').decode('utf-8')

html = f'''
<div style="display: flex; justify-content: space-around; flex-wrap: wrap; gap: 20px; align-items: flex-start;">
    <div>{moore_svg}</div>
    <div>{mealy_svg}</div>
</div>
'''
display(HTML(html))

**Key observations:**

| Aspect | Moore | Mealy |
|--------|-------|-------|
| States needed | 3 | 2 |
| Output location | Inside states | On transitions |
| Output timing | Changes on clock edge | Can change with input |
| Notation | State/Output | Input/Output on arrows |

The Mealy machine needs fewer states because the output depends on both state AND input, allowing more information per state.

---

## State Tables: The Formal Specification

State diagrams are visual, but state tables provide a complete, unambiguous specification that maps directly to hardware.

### Moore Machine State Table

For the sequence detector Moore machine:

| Current State | Input (X) | Next State | Output (Z) |
|--------------|-----------|------------|------------|
| S0           | 0         | S0         | 0          |
| S0           | 1         | S1         | 0          |
| S1           | 0         | S0         | 0          |
| S1           | 1         | S2         | 0          |
| S2           | 0         | S0         | 1          |
| S2           | 1         | S2         | 1          |

**How to read this table (Moore machine):**

- **Output column**: Shows the value *while in the current state* (before the clock edge)
- **Next State column**: Shows where we go *after the clock edge*
- **Key timing**: Output changes only when state changes (on clock edge)

**Example:** Row 5 shows in state S2 with input X=0, output Z=1 now, next state is S0 (where Z will become 0 after the clock edge).

### Mealy Machine State Table

For the Mealy version:

| Current State | X=0 (Next/Out) | X=1 (Next/Out) |
|--------------|----------------|----------------|
| S0           | S0/0           | S1/0           |
| S1           | S0/0           | S1/1           |

**How to read this table (Mealy machine):**

- **Notation**: "NextState/Output" shows both next state and output for that input
- **Output timing**: Output is produced *combinationally* (changes as soon as input changes, independent of the clock)
- **Key difference**: Output can change *before* the clock edge when input changes (asynchronous to state transitions)

**Example:** Row 2, column "X=1" shows "S1/1" means: currently in S1 with input X=1, output Z=1 is produced **combinationally** (immediately when X becomes 1, not waiting for a clock edge), and we'll stay in S1 after the next clock edge.

**Moore vs Mealy timing:**
- **Moore**: Output changes only on clock edges (when state changes) — synchronous
- **Mealy**: Output changes immediately when input changes (combinational logic) — can be asynchronous to clock

---

## State Encoding

To implement an FSM in hardware, we need to assign binary codes to states. The number of flip-flops required is:

$$\text{Number of flip-flops} = \lceil \log_2(\text{number of states}) \rceil$$

For our 3-state Moore machine: $\lceil \log_2(3) \rceil = 2$ flip-flops.

**Why?** 2 flip-flops can represent $2^2 = 4$ distinct states, which is the minimum needed for 3 states. With only 1 flip-flop we'd have $2^1 = 2$ states (insufficient). The 4th state (binary 11) will be unused in our design—we call this a "don't care" state.

### Common Encoding Schemes

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch

fig, ax = plt.subplots(figsize=(12, 5))
ax.axis('off')
ax.set_xlim(0, 100)
ax.set_ylim(0, 60)
ax.set_title('State Encoding Schemes for 4 States', fontsize=14, fontweight='bold', pad=15)

# Binary encoding
binary_box = FancyBboxPatch((5, 25), 25, 30, boxstyle='round,pad=0.02',
                            facecolor='#dbeafe', edgecolor='#3b82f6', linewidth=2)
ax.add_patch(binary_box)
ax.text(17.5, 52, 'Binary', ha='center', fontsize=11, fontweight='bold', color='#1e40af')
ax.text(17.5, 45, 'S0 = 00', ha='center', fontsize=10, family='monospace')
ax.text(17.5, 40, 'S1 = 01', ha='center', fontsize=10, family='monospace')
ax.text(17.5, 35, 'S2 = 10', ha='center', fontsize=10, family='monospace')
ax.text(17.5, 30, 'S3 = 11', ha='center', fontsize=10, family='monospace')
ax.text(17.5, 22, '2 flip-flops', ha='center', fontsize=9, color='#6b7280')

# Gray code
gray_box = FancyBboxPatch((37, 25), 25, 30, boxstyle='round,pad=0.02',
                          facecolor='#d1fae5', edgecolor='#10b981', linewidth=2)
ax.add_patch(gray_box)
ax.text(49.5, 52, 'Gray Code', ha='center', fontsize=11, fontweight='bold', color='#065f46')
ax.text(49.5, 45, 'S0 = 00', ha='center', fontsize=10, family='monospace')
ax.text(49.5, 40, 'S1 = 01', ha='center', fontsize=10, family='monospace')
ax.text(49.5, 35, 'S2 = 11', ha='center', fontsize=10, family='monospace')
ax.text(49.5, 30, 'S3 = 10', ha='center', fontsize=10, family='monospace')
ax.text(49.5, 22, '2 flip-flops', ha='center', fontsize=9, color='#6b7280')

# One-hot
onehot_box = FancyBboxPatch((69, 25), 25, 30, boxstyle='round,pad=0.02',
                            facecolor='#fef3c7', edgecolor='#f59e0b', linewidth=2)
ax.add_patch(onehot_box)
ax.text(81.5, 52, 'One-Hot', ha='center', fontsize=11, fontweight='bold', color='#92400e')
ax.text(81.5, 45, 'S0 = 0001', ha='center', fontsize=10, family='monospace')
ax.text(81.5, 40, 'S1 = 0010', ha='center', fontsize=10, family='monospace')
ax.text(81.5, 35, 'S2 = 0100', ha='center', fontsize=10, family='monospace')
ax.text(81.5, 30, 'S3 = 1000', ha='center', fontsize=10, family='monospace')
ax.text(81.5, 22, '4 flip-flops', ha='center', fontsize=9, color='#6b7280')

ax.text(50, 10, 'Binary: Minimum flip-flops, complex next-state logic', ha='center', fontsize=9, color='#374151')
ax.text(50, 5, 'One-Hot: More flip-flops, but simpler combinational logic (common in FPGAs)', ha='center', fontsize=9, color='#374151')

plt.tight_layout()
plt.show()

**Choosing an encoding:**

- **Binary**: Minimizes flip-flops; good for resource-constrained designs
- **Gray code**: Adjacent states differ by one bit; reduces glitches in outputs
- **One-hot**: One flip-flop per state; simplifies next-state logic, common in FPGAs

---

## Hardware Implementation with D Flip-Flops

Let's implement our Moore sequence detector using D flip-flops and binary encoding:

| State | Q1 Q0 |
|-------|-------|
| S0    | 0  0  |
| S1    | 0  1  |
| S2    | 1  0  |

### Step 1: Encoded State Table

| Q1 | Q0 | X | D1 (Next Q1) | D0 (Next Q0) | Z |
|----|----|---|--------------|--------------|---|
| 0  | 0  | 0 | 0            | 0            | 0 |
| 0  | 0  | 1 | 0            | 1            | 0 |
| 0  | 1  | 0 | 0            | 0            | 0 |
| 0  | 1  | 1 | 1            | 0            | 0 |
| 1  | 0  | 0 | 0            | 0            | 1 |
| 1  | 0  | 1 | 1            | 0            | 1 |
| 1  | 1  | X | -            | -            | - |

**How to read this encoded state table:**

- **Q1, Q0 columns**: Current state encoded as binary flip-flop values (00=S0, 01=S1, 10=S2)
- **X column**: Input value at the clock edge
- **D1, D0 columns**: Next state encoded as flip-flop inputs (values loaded on the rising clock edge)
- **Z column**: Output produced *while in the current state* (Moore machine - depends only on Q1, Q0)
- **Row 7 (Q1=1, Q0=1)**: Unused state - we only need 3 states, so the 4th is a "don't care" (marked with X/-)

**Note:** This table directly maps symbolic states (S0, S1, S2) to hardware (flip-flop values). The Z output is combinational from Q1, Q0. The D1, D0 inputs will set the next state after the clock edge.

### Step 2: Derive Boolean Equations

Using Karnaugh maps or inspection:

$$D_1 = Q_0 \cdot X + Q_1 \cdot X = X \cdot (Q_0 + Q_1)$$

$$D_0 = \overline{Q_1} \cdot \overline{Q_0} \cdot X$$

$$Z = Q_1$$

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch

fig, ax = plt.subplots(figsize=(14, 8))
ax.axis('off')
ax.set_xlim(0, 140)
ax.set_ylim(0, 90)
ax.set_title('FSM Hardware Architecture', fontsize=14, fontweight='bold', pad=15)

ff_color = '#3b82f6'
logic_color = '#10b981'
wire_color = '#374151'

# Next-state logic box
ns_box = FancyBboxPatch((10, 30), 30, 35, boxstyle='round,pad=0.02',
                        facecolor='#d1fae5', edgecolor=logic_color, linewidth=2)
ax.add_patch(ns_box)
ax.text(25, 60, 'Next-State', ha='center', fontsize=11, fontweight='bold', color='#065f46')
ax.text(25, 54, 'Logic', ha='center', fontsize=11, fontweight='bold', color='#065f46')
ax.text(25, 45, 'D₁ = X(Q₀+Q₁)', ha='center', fontsize=9, family='monospace', color='#065f46')
ax.text(25, 39, 'D₀ = X·Q̄₁·Q̄₀', ha='center', fontsize=9, family='monospace', color='#065f46')

# D Flip-flop 1
ff1_box = FancyBboxPatch((55, 50), 20, 18, boxstyle='round,pad=0.02',
                         facecolor='#dbeafe', edgecolor=ff_color, linewidth=2)
ax.add_patch(ff1_box)
ax.text(65, 62, 'D FF', ha='center', fontsize=10, fontweight='bold', color='#1e40af')
ax.text(65, 56, 'Q₁', ha='center', fontsize=11, color='#1e40af')
ax.text(57, 59, 'D₁', ha='center', fontsize=9, color='#374151')

# D Flip-flop 0
ff0_box = FancyBboxPatch((55, 25), 20, 18, boxstyle='round,pad=0.02',
                         facecolor='#dbeafe', edgecolor=ff_color, linewidth=2)
ax.add_patch(ff0_box)
ax.text(65, 37, 'D FF', ha='center', fontsize=10, fontweight='bold', color='#1e40af')
ax.text(65, 31, 'Q₀', ha='center', fontsize=11, color='#1e40af')
ax.text(57, 34, 'D₀', ha='center', fontsize=9, color='#374151')

# Output logic box
out_box = FancyBboxPatch((95, 40), 25, 20, boxstyle='round,pad=0.02',
                         facecolor='#fef3c7', edgecolor='#f59e0b', linewidth=2)
ax.add_patch(out_box)
ax.text(107.5, 55, 'Output', ha='center', fontsize=11, fontweight='bold', color='#92400e')
ax.text(107.5, 49, 'Logic', ha='center', fontsize=11, fontweight='bold', color='#92400e')
ax.text(107.5, 44, 'Z = Q₁', ha='center', fontsize=9, family='monospace', color='#92400e')

# Input X
ax.annotate('', xy=(10, 47), xytext=(0, 47), arrowprops=dict(arrowstyle='->', color=wire_color, lw=2))
ax.text(2, 50, 'X', ha='center', fontsize=12, fontweight='bold')

# Clock
ax.annotate('', xy=(65, 25), xytext=(65, 15), arrowprops=dict(arrowstyle='->', color='#ef4444', lw=2))
ax.text(65, 12, 'CLK', ha='center', fontsize=10, color='#ef4444', fontweight='bold')

# Wires
ax.plot([40, 55], [55, 59], color=wire_color, lw=1.5)
ax.plot([40, 55], [40, 34], color=wire_color, lw=1.5)
ax.plot([75, 95], [59, 50], color=wire_color, lw=1.5)

# Feedback
ax.plot([75, 85, 85, 5, 5, 10], [59, 59, 75, 75, 52, 52], color='#8b5cf6', lw=1.5, linestyle='--')
ax.plot([75, 88, 88, 3, 3, 10], [34, 34, 78, 78, 42, 42], color='#8b5cf6', lw=1.5, linestyle='--')
ax.text(45, 80, 'Feedback (current state)', ha='center', fontsize=9, color='#8b5cf6', style='italic')

# Output Z
ax.annotate('', xy=(135, 50), xytext=(120, 50), arrowprops=dict(arrowstyle='->', color=wire_color, lw=2))
ax.text(132, 53, 'Z', ha='center', fontsize=12, fontweight='bold')

ax.text(70, 5, 'FSM = Combinational Logic (next-state + output) + Sequential Elements (flip-flops)',
        ha='center', fontsize=10, style='italic', color='#6b7280')

plt.tight_layout()
plt.show()

### The General FSM Architecture

Every synchronous FSM follows this pattern:

1. **State Register**: Flip-flops store the current state (Q values)
2. **Next-State Logic**: Combinational circuit computing D inputs from current state and inputs
3. **Output Logic**: Combinational circuit computing outputs
   - Moore: Output depends only on Q (current state)
   - Mealy: Output depends on Q and inputs
4. **Clock**: Synchronizes state transitions

---

## Complete Example: Traffic Light Controller

Let's design a simplified traffic light controller for a single intersection. The light cycles through:
- **Green** (30 time units)
- **Yellow** (5 time units)  
- **Red** (30 time units)

For simplicity, we'll use a `timer_done` input signal that goes high when the current phase should end.

In [ ]:
from graphviz import Digraph
from IPython.display import display, SVG

dot = Digraph('traffic_light')
dot.attr(rankdir='LR')
dot.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='11', width='0.9')
dot.attr('edge', fontname='Arial', fontsize='9')

# States with traffic light colors
dot.node('GREEN', 'GREEN\nR=0,Y=0,G=1', fillcolor='#22c55e', fontcolor='white')
dot.node('YELLOW', 'YELLOW\nR=0,Y=1,G=0', fillcolor='#eab308', fontcolor='#422006')
dot.node('RED', 'RED\nR=1,Y=0,G=0', fillcolor='#ef4444', fontcolor='white')

# Initial state
dot.node('reset', '', shape='point', width='0')
dot.edge('reset', 'GREEN', label='Reset')

# Transitions (timer_done=1)
dot.edge('GREEN', 'YELLOW', label='T=1', penwidth='2')
dot.edge('YELLOW', 'RED', label='T=1', penwidth='2')
dot.edge('RED', 'GREEN', label='T=1', penwidth='2')

# Self-loops (timer_done=0)
dot.edge('GREEN', 'GREEN', label='T=0', color='#9ca3af', fontcolor='#6b7280')
dot.edge('YELLOW', 'YELLOW', label='T=0', color='#9ca3af', fontcolor='#6b7280')
dot.edge('RED', 'RED', label='T=0', color='#9ca3af', fontcolor='#6b7280')

display(SVG(dot.pipe(format='svg')))

### State Table for Traffic Light Controller

We'll use **binary encoding** for the three states:

| State | Q1 Q0 |
|-------|-------|
| GREEN  | 0  0  |
| YELLOW | 0  1  |
| RED    | 1  0  |

The complete state table with encoded states:

| Current State | Q1 Q0 | timer_done | Next State | D1 D0 | R | Y | G |
|--------------|-------|------------|------------|-------|---|---|---|
| GREEN        | 0  0  | 0          | GREEN      | 0  0  | 0 | 0 | 1 |
| GREEN        | 0  0  | 1          | YELLOW     | 0  1  | 0 | 0 | 1 |
| YELLOW       | 0  1  | 0          | YELLOW     | 0  1  | 0 | 1 | 0 |
| YELLOW       | 0  1  | 1          | RED        | 1  0  | 0 | 1 | 0 |
| RED          | 1  0  | 0          | RED        | 1  0  | 1 | 0 | 0 |
| RED          | 1  0  | 1          | GREEN      | 0  0  | 1 | 0 | 0 |

**Note:** R, Y, G columns show outputs *while in the current state* (Moore machine). D1, D0 are the flip-flop inputs that will set the next state on the clock edge.

### Boolean Equations

From the state table:

$$D_1 = Q_0 \cdot T + Q_1 \cdot \overline{T}$$

$$D_0 = \overline{Q_1} \cdot \overline{Q_0} \cdot T + Q_0 \cdot \overline{T}$$

$$R = Q_1, \quad Y = Q_0, \quad G = \overline{Q_1} \cdot \overline{Q_0}$$

where $T$ = timer_done.

---

## Timing Analysis

Understanding FSM timing is critical for correct operation. Let's trace through our sequence detector.

**About the input sequence:** The X pattern `[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0]` was carefully chosen to demonstrate:

1. **First detection (cycles 1-3)**: Two consecutive 1s → S0→S1→S2, Z becomes 1
2. **Reset mid-sequence (cycle 3-4)**: X=0 breaks the streak → back to S0
3. **Second detection (cycles 4-7)**: Three consecutive 1s → detects at cycle 6, stays in S2 for cycle 7
4. **Double reset (cycles 7-8)**: Two 0s in a row → confirms we're back at S0
5. **Third detection (cycles 9-11)**: Another pair of 1s → detection at cycle 11

**Why this pattern is instructive:**
- Shows multiple detections in one trace
- Demonstrates that Z=1 persists while X=1 continues (cycles 6-7)
- Shows immediate reset to S0 on X=0 from any state
- The 0s at different points show the FSM properly restarting

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(4, 1, figsize=(14, 8), sharex=True)
fig.suptitle('Timing Diagram: Sequence Detector (Moore Machine)', fontsize=14, fontweight='bold')

# Time points
t = np.arange(0, 12, 0.01)

# Clock signal
clk = np.zeros_like(t)
for i in range(12):
    clk[(t >= i) & (t < i + 0.5)] = 1

# Input X signal: 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0
x_values = [0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0]
x_signal = np.zeros_like(t)
for i, val in enumerate(x_values):
    x_signal[(t >= i) & (t < i + 1)] = val

# State sequence: S0->S0->S1->S2->S0->S1->S2->S2->S0->S0->S1->S2
# State at each clock: after seeing input
state_values = [0, 0, 1, 2, 0, 1, 2, 2, 0, 0, 1, 2]  # 0=S0, 1=S1, 2=S2
state_signal = np.zeros_like(t)
for i, val in enumerate(state_values):
    state_signal[(t >= i) & (t < i + 1)] = val

# Output Z (Moore: depends only on state)
z_signal = np.zeros_like(t)
for i, val in enumerate(state_values):
    z_signal[(t >= i) & (t < i + 1)] = 1 if val == 2 else 0

# Plot clock
axes[0].fill_between(t, 0, clk, color='#3b82f6', alpha=0.3, step='pre')
axes[0].plot(t, clk, color='#3b82f6', lw=2, drawstyle='steps-pre')
axes[0].set_ylabel('CLK', fontsize=11, fontweight='bold')
axes[0].set_ylim(-0.2, 1.4)
axes[0].set_yticks([0, 1])
axes[0].grid(True, alpha=0.3, axis='x')

# Plot X
axes[1].fill_between(t, 0, x_signal, color='#10b981', alpha=0.3, step='pre')
axes[1].plot(t, x_signal, color='#10b981', lw=2, drawstyle='steps-pre')
axes[1].set_ylabel('X (Input)', fontsize=11, fontweight='bold')
axes[1].set_ylim(-0.2, 1.4)
axes[1].set_yticks([0, 1])
axes[1].grid(True, alpha=0.3, axis='x')

# Plot State
colors = ['#6b7280', '#8b5cf6', '#f59e0b']
labels = ['S0', 'S1', 'S2']
axes[2].plot(t, state_signal, color='#374151', lw=2, drawstyle='steps-pre')
for i in range(12):
    state = state_values[i]
    axes[2].fill_between([i, i+1], 0, state, color=colors[state], alpha=0.4, step='pre')
    axes[2].text(i + 0.5, state + 0.15, labels[state], ha='center', fontsize=9, fontweight='bold')
axes[2].set_ylabel('State', fontsize=11, fontweight='bold')
axes[2].set_ylim(-0.3, 2.6)
axes[2].set_yticks([0, 1, 2])
axes[2].set_yticklabels(['S0', 'S1', 'S2'])
axes[2].grid(True, alpha=0.3, axis='x')

# Plot Z
axes[3].fill_between(t, 0, z_signal, color='#ef4444', alpha=0.3, step='pre')
axes[3].plot(t, z_signal, color='#ef4444', lw=2, drawstyle='steps-pre')
axes[3].set_ylabel('Z (Output)', fontsize=11, fontweight='bold')
axes[3].set_ylim(-0.2, 1.4)
axes[3].set_yticks([0, 1])
axes[3].set_xlabel('Clock Cycles', fontsize=11)
axes[3].grid(True, alpha=0.3, axis='x')

# Add clock edge markers
for i in range(12):
    for ax in axes:
        ax.axvline(x=i, color='#d1d5db', linestyle='--', lw=0.5)

# Highlight detections
for i in [3, 6, 7, 11]:
    axes[3].annotate('Detected!', xy=(i + 0.5, 1.1), fontsize=8, ha='center', color='#ef4444')

plt.tight_layout()
plt.show()

**Reading the timing diagram:**

1. State changes occur at the **rising edge** of the clock
2. The new state depends on the **current state** and **input at the clock edge**
3. In a Moore machine, output Z changes only when the state changes
4. Detection (Z=1) occurs when we reach state S2, which happens after seeing two consecutive 1s

---

## Common FSM Design Patterns

### Pattern 1: Sequence Detector
Detects a specific bit pattern in a stream. We built one above.

### Pattern 2: Counter
Cycles through states in order. Each state represents a count value.

### Pattern 3: Controller
Orchestrates multi-step operations. States represent phases of an operation (like the traffic light).

### Pattern 4: Arbiter
Manages access to shared resources. States track which requester has access.

In [ ]:
from graphviz import Digraph
from IPython.display import display, HTML

# === 3-bit Counter ===
counter = Digraph('counter')
counter.attr(label='3-Bit Counter FSM\nCounts: 000 → 001 → ... → 111 → 000', labelloc='t', fontsize='12')
counter.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='10', width='0.6', fillcolor='#3b82f6', fontcolor='white')
counter.attr('edge', fontname='Arial', fontsize='8')

for i in range(8):
    counter.node(f'C{i}', f'{i}\n{i:03b}')

counter.node('start_c', '', shape='point', width='0')
counter.edge('start_c', 'C0', label='Reset')

for i in range(8):
    counter.edge(f'C{i}', f'C{(i+1)%8}')

# === Arbiter ===
arbiter = Digraph('arbiter')
arbiter.attr(rankdir='TB', label='Simple 2-Request Arbiter\nPriority: A > B', labelloc='t', fontsize='12')
arbiter.attr('node', shape='circle', style='filled', fontname='Arial', fontsize='10', width='0.7')
arbiter.attr('edge', fontname='Arial', fontsize='9')

arbiter.node('IDLE', 'IDLE', fillcolor='#6b7280', fontcolor='white')
arbiter.node('GRANT_A', 'GRANT\nA', fillcolor='#3b82f6', fontcolor='white')
arbiter.node('GRANT_B', 'GRANT\nB', fillcolor='#10b981', fontcolor='white')
arbiter.node('start_a', '', shape='point', width='0')
arbiter.edge('start_a', 'IDLE', label='Reset')
arbiter.edge('IDLE', 'GRANT_A', label='reqA=1', penwidth='1.5')
arbiter.edge('IDLE', 'GRANT_B', label='reqĀ·reqB', penwidth='1.5')
arbiter.edge('GRANT_A', 'IDLE', label='done', color='#9ca3af', fontcolor='#6b7280')
arbiter.edge('GRANT_B', 'IDLE', label='done', color='#9ca3af', fontcolor='#6b7280')

# Get SVG strings and embed in a single HTML output
counter_svg = counter.pipe(format='svg').decode('utf-8')
arbiter_svg = arbiter.pipe(format='svg').decode('utf-8')

html = f'''
<div style="display: flex; justify-content: space-around; flex-wrap: wrap; gap: 20px; align-items: flex-start;">
    <div>{counter_svg}</div>
    <div>{arbiter_svg}</div>
</div>
'''
display(HTML(html))

---

## Key Takeaways

1. **FSMs are the foundation** of sequential digital design — nearly every digital system contains FSMs

2. **Moore vs Mealy** is a fundamental design choice:
   - Moore: Simpler timing, outputs change only on clock edges
   - Mealy: Potentially fewer states, faster response to inputs

3. **State encoding** affects implementation complexity:
   - Binary minimizes flip-flops
   - One-hot simplifies combinational logic

4. **The general architecture** is always:
   - State register (flip-flops)
   - Next-state combinational logic
   - Output combinational logic

5. **Design process**:
   1. Draw state diagram from requirements
   2. Create state table
   3. Choose state encoding
   4. Derive Boolean equations
   5. Implement with gates and flip-flops (or HDL)

---

*FSMs are everywhere in digital design — from simple button debouncers to complex CPU control units. Master them, and you have a powerful tool for any sequential logic problem.*